# Setup


# Define a hello world GUI plugin

The example below use basic HTML/Javascript/CSS to build a simple hellow world example. If you are not familiar with web development, there are good documentation and tutorials here: https://www.w3schools.com/

In [ ]:
## ImJoy Plugin
from IPython.display import HTML
my_plugin_source = HTML('''
<docs lang="markdown">
[TODO: write documentation for this plugin.]
</docs>

<config lang="json">
{
  "name": "Untitled Plugin",
  "type": "window",
  "tags": [],
  "ui": "",
  "version": "0.1.0",
  "cover": "",
  "description": "[TODO: describe this plugin with one sentence.]",
  "icon": "extension",
  "inputs": null,
  "outputs": null,
  "api_version": "0.1.8",
  "env": "",
  "permissions": [],
  "requirements": [],
  "dependencies": [],
  "defaults": {"w": 20, "h": 10}
}
</config>

<script lang="javascript">
class ImJoyPlugin {
  async setup() {
    api.log('initialized')
  }

  async run(ctx) {

  }
}

api.export(new ImJoyPlugin())
</script>

<window lang="html">
  <div>
    <h2>
    Hello, ImJoy
    </h2>
  </div>
</window>

<style lang="css">

</style>
''')

# Use the hello world GUI plugin

In [ ]:
from imjoy import api

class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        # now use show a window with the plugin defined above
        # viewer = await api.createWindow(src=my_plugin_source)
        # if you want to popup dialog, call:
        viewer = await api.showDialog(src=my_plugin_source)

api.export(ImJoyPlugin())

# Advanced example: Compare Images plugin

## Step 1 define a plugin

In [ ]:
## ImJoy Plugin
from IPython.display import HTML
compare_plugin_source = HTML('''

<config lang="json">
{
  "name": "CompareImages",
  "type": "window",
  "tags": [],
  "ui": "",
  "version": "0.1.3",
  "cover": "",
  "description": "Compare images with two itk-vtk-viewers",
  "icon": "extension",
  "inputs": null,
  "outputs": null,
  "api_version": "0.1.8",
  "env": "",
  "permissions": [],
  "requirements": ["https://oeway.github.io/itk-vtk-viewer/itkVtkViewerCDN.js"],
  "dependencies": [],
  "defaults": {"w": 20, "h": 10, "fullscreen": true}
}
</config>

<script lang="javascript">
api.registerCodec({
  name: 'itkimage',
  decoder: itkVtkViewer.utils.convertToItkImage
})
api.registerCodec({
  name: 'ndarray',
  decoder: itkVtkViewer.utils.ndarrayToItkImage
})

function convertImageUrl2Itk(url){
  return new Promise((resolve, reject)=>{
      const canvas = document.createElement("canvas");
      const ctx = canvas.getContext("2d");
      const image = new Image();
      image.onload = function() {
        canvas.width=image.width;
        canvas.height=image.height;
        ctx.drawImage(image, 0, 0, image.width, image.height);
        const imageData = ctx.getImageData(0, 0, image.width, image.height);
        const vtkImage = itkVtkViewer.utils.vtkITKHelper.convertItkToVtkImage({
          imageType: { dimension: 2, pixelType: 1, componentType: 'uint8_t', components: 4},
          name: 'test image',
          origin: [0,0],
          spacing: [1,1],
          direction:{data: [1,0,0,1]},
          size: [image.width, image.height],
          data: new Uint8Array(imageData.data.buffer)
        })
        resolve(vtkImage);
      };
      image.crossOrigin = "Anonymous";
      image.src = url
  })
}

const containerStyle = {
  position: 'relative',
  width: '100%',
  height: '600px',
  minHeight: '400px',
  minWidth: '400px',
  margin: '1',
  padding: '1',
  top: '0',
  left: '0',
  overflow: 'hidden',
  display: 'block-inline'
};
const viewerStyle = {
  backgroundColor: [1.0, 1.0, 1.0],
  containerStyle: containerStyle,
};

async function imshow(imageData, el){
    if(typeof imageData === 'string'){
      imageData = await convertImageUrl2Itk(imageData)
    }
    else{
      imageData = itkVtkViewer.utils.vtkITKHelper.convertItkToVtkImage(imageData)
    }
    // clear container
    el.innerHTML = "";
    const toolbar = document.createElement('div')
    el.appendChild(toolbar)
    const view = document.createElement('div')
    el.appendChild(view)
    const dims = imageData.getDimensions()
    const is2D = dims.length === 2 || (dims.length === 3 && dims[2] === 1)
    const viewer = itkVtkViewer.createViewer(view, {
      viewerStyle: viewerStyle,
      image: imageData,
      pointSets: null,
      geometries: null,
      use2D: is2D,
      rotate: false,
      uiContainer: toolbar
    })
    viewer.setUserInterfaceCollapsed(true)
    return viewer
}

// synchronize the viewers
function syncViewers(viewers){
  const camera = viewers[0].getViewProxy().getCamera()
  // use the same camera
  for(let i=1;i<viewers.length;i++)
  viewers[i].getViewProxy().getRenderer().setActiveCamera(camera)
  camera.onModified((evt)=>{
    for(let i=0;i<viewers.length;i++)
    viewers[i].getViewProxy().getRenderWindow().render()
  })
}

class ImJoyPlugin {
  async setup() {
    api.log('initialized')
  }

  async compare(images) {
    const root = document.getElementById('app')
    root.innerHTML = ""
    const viewers = []
    for(let i=0;i<images.length;i++){
      const elm = document.createElement('div');      
      root.appendChild(elm)
      elm.classList.add('viewer-container')
      elm.style.width = 96/images.length + '%'
      elm.style.border = "1px solid #a6c6fc"
      const viewer = await imshow(images[i], elm)
      viewers.push(viewer)
    }
    syncViewers(viewers)
  }
  async run(ctx){
    if(ctx.data && ctx.data.images){
      this.compare(ctx.data.images)
    }
    else{
      this.compare(['https://images.proteinatlas.org/19661/221_G2_1_red_green.jpg', 'https://images.proteinatlas.org/19661/221_G2_1_yellow.jpg', 'https://images.proteinatlas.org/19661/221_G2_1_blue.jpg'])
    }
  }
}

api.export(new ImJoyPlugin())
</script>

<window lang="html">
  <div id="app">
  </div>
</window>

<style lang="css">
#app{
  display: table;
}
.viewer-container{
  display: table-cell;
  position: relative;
}
</style>
''')

## Step 2 Load the plugin from source code

We have stored the source code of a GUI plugin in `compare_plugin_source`, now we can use it via 
`await api.createWindow(src=compare_plugin_source)`.

In [ ]:
from imjoy import api
import skimage
import numpy as np

class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        volume = np.random.randint(0, 255, [30, 30, 30], dtype='uint8')
        # load the image compare window plugin from source code
        viewer = await api.createWindow(src=compare_plugin_source, data={"images": [volume, volume]})

api.export(ImJoyPlugin())

# Deploy the plugin

You can now deploy the plugin and access it via a url. The easiest way to deploy is to upload to a Github repo or [Gist](https://gist.github.com/).

For example, we can copy and paste the plugin code string (remove the `HTML()` and quotes around it) to gist: https://gist.github.com/oeway/ffb6f0efae8a68d497202137820f68e8

And use it as the following:

In [ ]:
from imjoy import api
import skimage
import numpy as np

class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        volume = np.random.randint(0, 255, [30, 30, 30], dtype='uint8')
        # load the image compare window plugin from url
        compare_plugin_url = "https://gist.github.com/oeway/ffb6f0efae8a68d497202137820f68e8"
        viewer = await api.createWindow(src=compare_plugin_url, data={"images": [volume, volume]})

api.export(ImJoyPlugin())

# Building frontend with Vue.js and Buefy

With the basic window plugin example above, it should give you a quick start. However, you may feel handcrafting the UI is a bit painful and tedious. In fact, there are many existing libries in Javascript or CSS can make your life easier. For example, [Vue.js](https://vuejs.org/) is a popular javascript framework and Buefy is a library that uses vuejs and [bulma](https://bulma.io/) to ease the processing of building GUI for your plugin.


Please take a look at the following example. 

You can add nice looking UI element, by copying code from buefy website. For example, if you want buttons, go to https://buefy.org/documentation/button, click `show code` in the example, and you can get `<b-button @click="clickMe">Click Me</b-button>`.

In [ ]:
## ImJoy Plugin
from IPython.display import HTML
plugin_source = HTML('''
<docs lang="markdown">
[TODO: write documentation for this plugin.]
</docs>

<config lang="json">
{
  "name": "BuefyBackbone",
  "type": "window",
  "tags": [],
  "ui": "",
  "version": "0.1.2",
  "api_version": "0.1.5",
  "description": "[TODO: add description]",
  "icon": "extension",
  "inputs": null,
  "outputs": null,
  "env": "",
  "requirements": [
      "https://cdnjs.cloudflare.com/ajax/libs/vue/2.5.22/vue.min.js",
      "https://unpkg.com/buefy/dist/buefy.min.css",
      "https://unpkg.com/buefy/dist/buefy.min.js",
      "https://cdn.materialdesignicons.com/5.3.45/css/materialdesignicons.min.css"
  ],
  "dependencies": [],
  "defaults": {"w": 20, "h": 10},
  "runnable": true
}
</config>

<script lang="javascript">

const app = new Vue({
  el: '#app',
  data: {
    title: 'Hello World'
  },
  methods: {
    async sayHi(){
      await api.alert('Hi')
    }
  }
})

class ImJoyPlugin {
  async setup() {

  }

  async run(my) {

  }
}

api.export(new ImJoyPlugin())
</script>

<window lang="html">
  <div id="app">
    <h1>{{title}}</h1>
    <b-button @click="sayHi()" type="is-primary">Say Hi</b-button>
  </div>
</window>

<style lang="css">
</style>
''')

from imjoy import api

class ImJoyPlugin():
    async def setup(self):
        pass

    async def run(self, ctx):
        viewer = await api.createWindow(src=plugin_source)

api.export(ImJoyPlugin())